In [1]:
import pandas as pd
import numpy as np

from glob import glob
from tqdm import tqdm
from scipy import interpolate

import warnings; warnings.filterwarnings(action='ignore') # 경고 메시지 무시
import matplotlib.pyplot as plt # 데이터 시각화 라이브러리
import pickle # 객체 입출력을 위한 라이브러리
from sklearn.model_selection import train_test_split # 훈련 데이터, 테스트 데이터 분리
from sklearn.preprocessing import StandardScaler # 정규화
from sklearn.ensemble import RandomForestClassifier as RFC # 랜덤포레스트 분류 알고리즘
from sklearn.tree import DecisionTreeClassifier as DTC # 의사결정나무 분류 알고리즘
from sklearn.ensemble import GradientBoostingClassifier as GBC # 그래디언트 부스팅 분류 알고리즘

In [2]:
w_list = sorted(glob("C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data/*.csv"))
w_list

['C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2012.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2013.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2014.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2015.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2016.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2017.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2018.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2019.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2020.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2021.csv',
 'C:/Users/liked/OneDrive/바탕 화면/데이콘/팔당댐/water_data\\data_2022.csv']

In [4]:
train_data = []
train_label = []
num = 0

for i in w_list[:-1]:
    
    tmp = pd.read_csv(i)
    tmp = tmp.replace(" ", np.nan)
    tmp = tmp.interpolate(method = 'values')
    tmp = tmp.fillna(0)
    
    # 431은 3일 간격을 의미하며, 432는 예측하고자 하는 시간을 의미
    
    for j in tqdm(range(len(tmp)-432)):
        train_data.append(np.array(tmp.loc[j:j + 431, ["swl", "inf", "sfw", "ecpc",
                                                       "tototf", "tide_level",
                                                       "fw_1018662", "fw_1018680",
                                                       "fw_1018683", "fw_1019630"]]).astype(float))
        
        train_label.append(np.array(tmp.loc[j + 432, ["wl_1018662", "wl_1018680",
                                                      "wl_1018683", "wl_1019630"]]).astype(float))

100%|██████████| 26064/26064 [00:37<00:00, 703.41it/s]


In [5]:
train_data = np.array(train_data)
train_label = np.array(train_label)
train_data

array([[[ 24.8  , 555.   , 219.07 , ...,   0.   , 729.8  , 540.18 ],
        [ 24.794, 464.6  , 218.86 , ...,   0.   , 731.48 , 540.18 ],
        [ 24.789, 478.1  , 218.69 , ...,   0.   , 726.42 , 540.18 ],
        ...,
        [ 25.039, 345.6  , 227.41 , ...,   0.   , 305.44 , 528.33 ],
        [ 25.039, 351.9  , 227.41 , ...,   0.   , 313.83 , 516.61 ],
        [ 25.039, 339.1  , 227.41 , ...,   0.   , 335.19 , 505.03 ]],

       [[ 24.794, 464.6  , 218.86 , ...,   0.   , 731.48 , 540.18 ],
        [ 24.789, 478.1  , 218.69 , ...,   0.   , 726.42 , 540.18 ],
        [ 24.789, 464.8  , 218.69 , ...,   0.   , 726.42 , 552.17 ],
        ...,
        [ 25.039, 351.9  , 227.41 , ...,   0.   , 313.83 , 516.61 ],
        [ 25.039, 339.1  , 227.41 , ...,   0.   , 335.19 , 505.03 ],
        [ 25.04 , 640.   , 227.44 , ...,   0.   , 343.1  , 505.03 ]],

       [[ 24.789, 478.1  , 218.69 , ...,   0.   , 726.42 , 540.18 ],
        [ 24.789, 464.8  , 218.69 , ...,   0.   , 726.42 , 552.17 ],
    